# Intro

I explored the music album sales data and got answers to questions below.

Which music genre was the best-selling in the US?

Which sales representitive achieved the best perfomance?

Who was the most popular artist in playlists?

To begin with, connected to the database and printed the first five rows to check the connection worked fine.

In [3]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [4]:
%%sql
SELECT * FROM customer
LIMIT 3;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3


### The best-selling genre in the US

I considered two things to find the best-selling genre in the US.

1. Count the invoice_line_id as a sales quantity.
   Join invoice_line, invoice, and customer tables to filter 'US' data.

2. Calculated how much each genre make up for the whole sales amount.

Top three genres, Rock, Alternative & Punk, and Metal account for over 80 percent. It is interesting that Rock music recorded overwhelming sales.

In [5]:
%%sql

WITH usa AS 
   (
   SELECT il.invoice_line_id, il.invoice_id, il.track_id
   FROM invoice_line il
   INNER JOIN invoice i ON i.invoice_id = il.invoice_id
   INNER JOIN customer c ON c.customer_id = i.customer_id
   WHERE c.country = "USA"
   )

    
SELECT
    g.genre_id,
    g.name AS gerne_name,
    COUNT(u.invoice_line_id) AS track_count,
    CAST(COUNT(u.invoice_line_id) AS FLOAT) / (
    SELECT COUNT(*) FROM usa) AS percentage
FROM usa u
INNER JOIN track t ON t.track_id = u.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 2
ORDER BY 3 DESC
LIMIT 10;

Done.


genre_id,gerne_name,track_count,percentage
1,Rock,561,0.5337773549000951
4,Alternative & Punk,130,0.12369172216936251
3,Metal,124,0.11798287345385347
14,R&B/Soul,53,0.05042816365366318
6,Blues,36,0.03425309229305423
23,Alternative,35,0.03330161750713606
7,Latin,22,0.02093244529019981
9,Pop,22,0.02093244529019981
17,Hip Hop/Rap,20,0.019029495718363463
2,Jazz,14,0.013320647002854425


### A sales representitive who achieved the best performance

Join employee, customer, and invoice tables.
Group by countries and sort them in descending order.

There are only three rows which meet all the conditions.
Jane Peachock working as a sales support agent in Canada ranks first.

In [6]:
%%sql

SELECT 
    e.employee_id,
    e.first_name||' '||e.last_name AS name,
    e.title,
    e.country,
    SUM(i.total) AS total_sales
FROM employee e
INNER JOIN customer c ON c.support_rep_id = e.employee_id
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY 1,3
ORDER BY 5 DESC;

Done.


employee_id,name,title,country,total_sales
3,Jane Peacock,Sales Support Agent,Canada,1731.5100000000039
4,Margaret Park,Sales Support Agent,Canada,1584.0000000000034
5,Steve Johnson,Sales Support Agent,Canada,1393.920000000002


### The most popluar artist in playlists

Join playlist_track, track, album, and artist tables.
Use the count function to calculate frequency of album_id alongside artist_name in each playlist_id.

After printing the top three rows, I found that Iron Maiden was the most loved musician.

In [8]:
%%sql
SELECT 
    p.playlist_id, 
    a.album_id, 
    a.title AS album_title,
    at.name AS artist_name,
    count(*)
FROM playlist_track p
INNER JOIN track t ON t.track_id = p.track_id
INNER JOIN album a ON a.album_id = t.album_id
INNER JOIN artist at ON at.artist_id = a.artist_id
GROUP BY a.artist_id
ORDER BY count(*) DESC
LIMIT 3;

Done.


playlist_id,album_id,album_title,artist_name,count(*)
17,111,Somewhere in Time,Iron Maiden,516
8,255,Instant Karma: The Amnesty International Campaign to Save Darfur,U2,333
17,154,Ride The Lightning,Metallica,296


This project was based on a learning material of Dataquest. But I wrote all the codes on my own.